In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import datetime
from matplotlib import pyplot as plt

%matplotlib inline
%run ./data_loading.ipynb

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv1a = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv3a = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=10*10*12, out_features=300)
        self.fc2 = nn.Linear(in_features=300, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=9)
        
    def forward(self, X):
        X = self.relu(self.conv1(X))
        X = self.relu(self.conv1a(X))
        X = self.pool(self.relu(self.conv2(X)))
        X = self.relu(self.conv3(X))
        X = self.relu(self.conv3a(X))
        X = self.pool(self.relu(self.conv4(X)))
        X = X.view(-1, 10*10*12)
        X = self.relu(self.fc1(X))
        X = self.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [3]:
def calculate_accuracy(dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc

In [4]:
def train(dataset, epochs, batch_size=5, to_cuda=True, calculate_acc=False):
    train_acc = []
    val_acc = []
    if to_cuda:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in tqdm(enumerate(dataloader, 0)):
            inputs, labels = data
            if to_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_f(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('epoch:', epoch, ', loss:', running_loss * batch_size / dataloader.__len__())
        if calculate_acc:
            t = calculate_accuracy(dataloader)
            v = calculate_accuracy(val_loader)
            print("train acc:", t, "val acc:", v)
            train_acc.append(t)
            val_acc.append(v)
        if epoch % 1 == 0: # every x epochs reshuffle data by creating a new loader
            if to_cuda:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
            else:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_acc, val_acc

In [5]:
extra_train = Parth_preprocessed_dataset
cr_test = ClassroomDataset(range_10=[7,10])
# Dividing data into 50% train, 40% val, 30% test
scores = []
for i in range(1):
    print("Fold number", i)
    net = Net().float()
    if torch.cuda.is_available():
        net = net.cuda()
    loss_f = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    if i == 0:
        cr_train = ClassroomDataset(range_10=[0,5])
        cr_val = ClassroomDataset(range_10=[5,7])
    else:
        cr_train = ClassroomDataset(range_10=[2,7])
        cr_val = ClassroomDataset(range_10=[0,2])
    acc = []
    for j in range(10):
        val_loader = DataLoader(cr_val, batch_size=1, shuffle=True)
        train(extra_train, epochs=4)
        print("val acc after extra:", calculate_accuracy(val_loader))
        train_accuracy, val_accuracy = train(cr_train, to_cuda=False, epochs=10, calculate_acc=True)
        print(train_accuracy, val_accuracy)
        acc.append((train_accuracy, val_accuracy))
    scores.append(acc)

Fold number 0


4575it [00:47, 95.42it/s]
10it [00:00, 90.92it/s]

epoch: 0 , loss: 9.735411584702998


4575it [00:47, 95.39it/s]
10it [00:00, 96.12it/s]

epoch: 1 , loss: 9.624203885187868


...

epoch: 8 , loss: 0.000243333728802142


13it [00:00, 120.68it/s]

train acc: 100.0 val acc: 94.6875


640it [00:05, 117.84it/s]


epoch: 9 , loss: 0.00022013336497384017
train acc: 100.0 val acc: 94.6875
[99.96875, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0] [94.453125, 94.765625, 94.6875, 94.6875, 94.6875, 94.6875, 94.6875, 94.6875, 94.6875, 94.6875]


In [6]:
scores

[[([93.90625,
    95.625,
    96.96875,
    97.5,
    97.3125,
    97.8125,
    99.25,
    98.1875,
    99.0625,
    99.5625],
   [90.234375,
    92.03125,
    91.484375,
    92.265625,
    92.03125,
    91.640625,
    93.28125,
    91.640625,
    92.734375,
    93.203125]),
  ([98.34375,
    99.28125,
    99.8125,
    98.96875,
    99.6875,
    99.46875,
    99.40625,
    99.90625,
    99.96875,
    99.96875],
   [92.578125,
    93.359375,
    93.75,
    93.046875,
    94.53125,
    93.046875,
    93.59375,
    93.828125,
    94.21875,
    93.828125]),
  ([99.78125,
    99.375,
    99.84375,
    100.0,
    99.96875,
    99.96875,
    99.96875,
    100.0,
    100.0,
    100.0],
   [93.984375,
    93.359375,
    94.0625,
    94.296875,
    94.21875,
    94.296875,
    94.375,
    94.375,
    94.296875,
    94.375]),
  ([99.4375, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0],
   [92.8125,
    93.671875,
    93.828125,
    93.984375,
    93.90625,
    93.828125,
    93.90

In [10]:
p = "weights/" + str(datetime.datetime.now()).split(".")[0].replace(':','_') + ".pth"
torch.save(net.state_dict(), p)
p

'weights/2019-12-02 13_47_01.pth'

In [8]:
def class_wise_accuracy(dataloader):
    correct = {}
    total = {}
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            for l in range(len(labels)):
                label = int(labels[l].cpu().numpy())
                pred = predicted[l]
                if label in correct:
                    correct[label] += int(pred == label)
                    total[label] += 1
                else:
                    correct[label] = int(pred == label)
                    total[label] = 1
    acc = {}
    for label in correct:
        acc[label] = 100 * correct[label] / total[label]
    return acc

In [9]:
class_wise_accuracy(val_loader)

{8: 96.31901840490798,
 7: 93.0379746835443,
 4: 96.29629629629629,
 2: 95.0920245398773,
 1: 92.40506329113924,
 3: 96.12903225806451,
 5: 91.41104294478528,
 6: 96.83544303797468}